<a href="https://colab.research.google.com/github/LouiseDupuis/PlateformeCodePhenix/blob/master/Sentiment_Analysis_with_CamemBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Loading Model

In [2]:
!pip install sentencepiece

     |████████████████████████████████| 1.1MB 4.7MB/s 


In [3]:
!pip install transformers

     |████████████████████████████████| 890kB 5.6MB/s 
     |████████████████████████████████| 3.0MB 33.8MB/s 
     |████████████████████████████████| 890kB 44.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=891b8ea985c6ec9ef7d4dff4dabd9515d4839152df1fb2936bb39a44b8e69123
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [4]:
from transformers import CamembertConfig, CamembertTokenizer, CamembertForSequenceClassification

# Initializing a configuration
configuration = CamembertConfig()

# Initializing a model from the configuration
model = CamembertForSequenceClassification.from_pretrained('camembert-base')
# Accessing the model configuration
configuration = model.config 





Some weights of the model checkpoint at camembert-base were not used when initializing CamembertForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing CamembertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing CamembertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at camembert-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'cl

# Preprocessing Data

BERT models need a special preprocessing of the data before input. Texts have to be tokenized, to be divided in separate sentences by special tokens and to be padded to a fixed length. 

In [14]:

sample_text = 'Ceci est un test.'

tokenizer = CamembertTokenizer.from_pretrained('camembert-base')


def encode_text(text_string):
  tensor = tokenizer.encode_plus(
  text_string,
  add_special_tokens=True, # Add '[CLS]' and '[SEP]'
  return_token_type_ids=False,
  pad_to_max_length=True,
  return_attention_mask=True,
  padding = 'max_length',
  return_tensors='pt',  # Return PyTorch tensors
  )
  return tensor 



In [15]:
print(encode_text(sample_text))

{'input_ids': tensor([[   5, 2978,   30,   23, 2006,    9,    6,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,

Creating a Pytorch Dataset, or save directly the preprocessed inputs as csv ?  

# Training the Model

The classifiers are already part of this model, but we could build our own on top of the basic CamemBERT model. 

From the CamemBERT original paper : 

We fine-tune independently CamemBERT for each task and each dataset. We optimise the model using the Adam optimiser [38] with a fixed learning rate. We run a grid search on a combination of learning rates and batch sizes. We select the best model on the validation set out of the 30 first epochs.

Although this might push the performances even further, for all tasks except NLI, we don’t apply any regularisation techniques such as weight decay, learning rate warm-up or discriminative fine-tuning. We show that fine-tuning CamemBERT in a straight-forward manner leads to state-of-the-art results on most tasks and outperforms the existing BERT-based models in most cases.

In [ ]:
# moving the model to GPU 

model = model.to(device)


# hyperparameters 

EPOCHS = 10
optimizer = AdamW(model.parameters(), lr=2e-5, correct_bias=False)
total_steps = len(train_data_loader) * EPOCHS
scheduler = get_linear_schedule_with_warmup(
  optimizer,
  num_warmup_steps=0,
  num_training_steps=total_steps
)





loss_fn = nn.CrossEntropyLoss().to(device)

